In [1]:
import pandas as pd
import numpy as np
import sys
import os

In [2]:
import winsound

In [3]:
os.listdir()

['.git',
 '.gitignore',
 '.ipynb_checkpoints',
 '2015_failures.csv',
 '2015_sample.csv',
 'boosting.ipynb',
 'data_Q1_2018_failures.csv',
 'data_Q2_2018_failures.csv',
 'data_Q3_2018_failures.csv',
 'hard_drive_intro.ipynb',
 'neural-nets.ipynb',
 'random_sample.py',
 'README.md',
 'sampling.ipynb',
 'save_failures.py']

In [4]:
good = pd.read_csv('2015_sample.csv')

In [5]:
bad = pd.read_csv('2015_failures.csv')

In [29]:
df = pd.concat([good,bad])

In [30]:
df = df.dropna(0,thresh=30).dropna(1)
columns = [i for i in df.columns if 'capacity' in i or 'normalized' in i]
X = df[columns]
y = df['failure']

In [8]:
import tensorflow as tf

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [31]:
X.head()

,capacity_bytes,smart_1_normalized,smart_3_normalized,smart_4_normalized,smart_5_normalized,smart_7_normalized,smart_9_normalized,smart_10_normalized,smart_12_normalized,smart_194_normalized,smart_197_normalized,smart_198_normalized,smart_199_normalized
0,3000592982016,100.0,141.0,100.0,100.0,100.0,98.0,100.0,100.0,222.0,100.0,100.0,200.0
1,4000787030016,116.0,93.0,100.0,100.0,84.0,92.0,100.0,100.0,27.0,100.0,100.0,200.0
2,4000787030016,119.0,91.0,100.0,100.0,87.0,78.0,100.0,100.0,19.0,100.0,100.0,200.0
3,4000787030016,117.0,95.0,100.0,100.0,84.0,84.0,100.0,100.0,25.0,100.0,100.0,200.0
4,3000592982016,100.0,140.0,100.0,100.0,100.0,97.0,100.0,100.0,193.0,100.0,100.0,200.0


In [27]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.66, random_state=42)

In [13]:
accuracy = lambda actual,predicted : 1- np.mean(np.abs(actual-predicted))

def precision(actual,predicted):
    tp = np.all([actual, predicted],axis=0).sum()
    fp = np.all([1-actual, predicted],axis=0).sum()
    tn = np.all([1-actual, 1-predicted],axis=0).sum()
    fn = np.all([actual, 1-predicted],axis=0).sum()
    return tp/(tp+fp) if (tp+fp>0) else 0

def recall(actual,predicted):
    tp = np.all([actual, predicted],axis=0).sum()
    fp = np.all([1-actual, predicted],axis=0).sum()
    tn = np.all([1-actual, 1-predicted],axis=0).sum()
    fn = np.all([actual, 1-predicted],axis=0).sum()
    return tp/(tp+fn) if (tp+fn>0) else 0

In [12]:
def layer(x, num_neuron, i, activation):
    input_shape = x.get_shape()
    W = tf.get_variable("weights_%d" %i, 
      [input_shape[-1], num_neuron], initializer = tf.truncated_normal_initializer(stddev = 0.1))
    b = tf.get_variable("bias_%d" %i, 
      [1, num_neuron], initializer = tf.truncated_normal_initializer(stddev = 0.1))
    y = tf.matmul(x, W) + b
    return activation(y)

In [33]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32, [None, X.shape[1]])
h = x
for i in range(3):
    h = layer(h,100,i,tf.nn.relu)
#output layer
h = layer(h,2,4,tf.nn.sigmoid)

In [34]:
prediction = h
y = h
y_ = tf.placeholder(tf.float32, [None, 2])
loss = tf.reduce_mean(tf.square(y - y_))
train_step = tf.train.GradientDescentOptimizer(.1).minimize(loss)

In [37]:
sess = tf.Session()
tf.global_variables_initializer().run(session=sess)
for i in range(1000):
    sess.run([loss, train_step], feed_dict={x: X_train, y_: [[i==0,i==1] for i in y_train]})

In [47]:
y_pred_ = sess.run(prediction, feed_dict={x:X_test})
y_pred = np.argmax(y_pred_,axis=1)

In [48]:
print("Trained: ","Neural net")
print("Precision:",precision(y_test,y_pred))
print("Recall:",recall(y_test,y_pred))
print("Accuracy:",accuracy(y_test,y_pred))

Trained:  Neural net
Precision: 0.4970822281167109
Recall: 1.0
Accuracy: 0.4970822281167109


In [49]:
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)